# Blokkon lévő adatok automatikus feldolgozása

## Bevezetés

Ebben a fejezetben ismertetem programom főbb jellemzőit, a megoldandó problémát, ezekhez kapcsolódó meglévő módszereket, alkalmazásokat és szolgáltatásokat. 

### Megoldandó probléma

A programom gépi látás témakörében mozog, amelyen belül  leginkább a kép szegmentációjával, az azon lévő adatok kinyerésével foglalkozom. Lényegében amikor nekiláttam ennek a projektnek, annyi lebegett a szemem előtt, hogy tudjam mikor, hol, mennyiért vásárálok, mindezt egy egyszerű webes felületen megjelenítve. Amíg a lefényképezett nyugtából kinyert adatok lesznek, sok folyamat lejátszódik a háttérben. Legfőbb probléma, hogy nagyon sokféle blokk létezik méretre és tartalomra vonatkozóan. Én ezért csak olyan magyarországon kiadható nyugtával foglalkozom amely POS terminálból származik és a tételek megtalálhatóak rajta. Szerencsére a Nemzeti Adó- és Vámhivatal a nyugta kiállítását alapvető szabályokhoz köti. Csak néhányat kiemelve ami számomra is fontos: a nyugta kibocsátásának kelte; a nyugta kibocsátójának neve és címe; a termék értékesítésének, szolgáltatás nyújtásának adót is tartalmazó ellenértéke. Mivel ezek minden blokkon megtalálhatók, ezért szűkül a megoldandó probléma .Röviden a megoldás úgy néz ki, hogy van egy lefényképezett blokk, melyen apróbb módosításokat végzek el először (nyújtás, vágás, küszöbölés), majd a letisztázott képet szegmentálom, mert csak bizonyos részek érdekelnek a blokk tartalmából, és ezután történik meg az optikai karakterfelismerés. Majd a kinyert adatokat egyszerű adatbázisban tárolom, amelyet ezután már könnyű megjeleníteni webes felületen. 

([Nyugta kötelező adattartalma - 17. oldal](https://www.nav.gov.hu/data/cms489948/18_A_szamla_nyugta_kibocsatasanak_alapvet__szabalyai_20190131.pdf))

### Kapcsolódó területek

ez lemaradt ezt is kiegészítem

### Módszerek, alkalmazások, szolgáltatások

Mivel ez nem egy új keletű dolog, már létezik számos ilyen megoldás, de azok általában mind zárt forráskódúak, nem láthatóak a háttérbeli folyamatok. Pár ilyen kész szoftvert az alábbiakban bemutatok, melyek többféle lehetőséget kínálnak. 

Az egyik ilyen alkalmazás a [Taggun](https://www.taggun.io/), amelyet ingyenesen ki lehet próbálni, viszont ha rendesen használni szeretnénk akkor 0.08 USD-be kerül blokkonként.. 82.26%-os pontossággal tud kinyerni adatokat ez az alkalmazás. Több nyelvet és több féle fájl formátumot tud kezelni. Gépi tanulásos módszert, a Google Vision API-t és Microsoft Cognitive Service API-t is használ az adatok kinyeréséhez.

A másik a [Tabscanner](https://tabscanner.com/). Ez a szoftver évek óta piacon jelen van. Különböző mesterséges intelligencia modellekkel támogatják a szolgáltatásukat. Ellenörző moduljuk segítségével nincs szükség emberi beavatkozásra, minden teljesen automatikusan működik. Kiválaszthatjuk milyen adatokkal szeretnénk csak foglalkozni és még ami nagyon kedvező ebben a termékben, hogy API-t biztosítanak a vevő részére, így bármilyen alkalmazásba bele lehet integrálni a megoldásukat. Naprakész adatbázissal rendelkezik a blokkok formátumára vonatkozóan

A harmadik amit még érdemes bemutatom az [Xtracta](https://xtracta.com/receipt-capture-api/)-nak a blokk olvasó API-ja. Ez egy mégnagyobb rendszer, melynek működését az alábbi ábrán láthatjuk. Egyszerű a működtetése, többféle helyről lehet blokkokat feltölteni (mobil alkalmazásuk is van), szintén biztosít API-t a programozók számára. Széles körben elterjedt szoftver, nagyobb cégek is használják (pl.: Volvo).

![Xtracta - folyamatábra](jupyter/xtracta-flow-diagram-desktop-receipts.png)

([Xtracta folyamatábra - kép](https://xtracta.com/wp-content/uploads/2017/11/xtracta-flow-diagram-desktop-receipts.png))


Erről majd még írok : https://github.com/mre/receipt-parser

## Mintaadatbázis

A következőkben megvizsgáljuk, hogy milyen típusú blokkokkal találkozhatunk manapság, milyen előfeltételek szükségesek a sikeres beolvasáshoz, a vizsgálandó adatoknak a szerkezetét, illetve az adatbázisunk felépítését.

### Nyugták fő típusai

Magyarországon kétféleképpen adhatunk nyugtát a vevőknek. Van a hagyományos kézzel kitöltött nyugta, mellyel nem foglalkozom a szoftveremben. A másik ilyen amivel mindennapokban talákozunk a POS terminálból nyomtatott nyugta. Ezen a típusú nyugtán kötelező feltüntetni a nyugta sorszámát,a nyugta kibocsátójának nevét, címét, adóazonosító számát, vagy a NAV által jóváhagyott más
azonosítót, a nyugta kibocsátásának keltét, illetve a fizetendő összeget. Az alábiakkban látható egy minta mely a SPAR Magyarország egyik üzletéből származik.

![Nyugta spar ](jupyter/nyugta.jpg)

Egészen sok adat található ezen a kis blokkon. Láthatjuk, hogy a fejlécben az eladóra vonatkozó adatok szerepelnek, majd a nyugta szó után a tételeket látjuk felsorolva, a fizetendő összeget, a fizetés módját, egyéb áruházi információt, tranzakció számát, pénztárosra vonatkozó adatokat, a nyugta számát, a nyugta kiadásának dátumát és a NAV ellenörző kódot. 



### Előfeltételek

Fontos tisztázni, hogy a képek a mintaadatbázisban telefonnal készült képek. Lehetne lapolvasót használni jobb minőség érdékben, de az lassab, körülményesebb munkafolyamatot eredményez. Mivel manapság majdnem minden ember zsebében ott lapul egy ilyen okostelefon, és egész jó minőségű képek készíthetőek, ezért maradtam ennél a beviteli módnál.  Egyszínű háttéren érdemes fényképet készíteni, mert az éldetektáló függvények problémát okoznak, ha valamilyen minta látszódik a nyugta környezetében. Viszonylag közelről készítem a képeket, és kerülöm a sötét helyeket. Az előző mintán jól látható, optimálisan meg van világítva és körübelül 30-40 cm magasságból készült a kép.

### Képre vonatkozó paraméterek

A telefonom 12 megpixeles képeket készít, melynek szélessége 3024 pixel magassága 4032 pixel, mindezt .jpg formátumban. Természetesen más formátumok és felbontások is megengedettek. Ha szeretnénk jó eredményeket elérni, akkor legalább 1500 pixel szélességű képpel érdemes próbálkozni, mert azon már szépen látszódnak a karakterek is. 

### Kinyerendő adatok szerkezete

A nyugtán ami információ elsősorban lényeges, az az eladó címe, melyet irányítószám, városnév, utcanév, házszám; a vásárlás dátuma melyet év, hónap, nap, óra, perc ; és a vásárlás összértéke melyet összeg pénznem formátumban tárolok adatbázisomban. 

Vesszővel elhatárolt adatszerkezetekben tárolom az adatokat. Minta blokkunkról a bejegyzés így néz ki:

```csv
3527 Miskolc Bajcsy-Zsilinkszky utca 2-4, 2019.08.13 07:30, 598 Ft
```

### Mintáink elnevezési és tárolási konvenciói